In [82]:
import ssl
import certifi
from urllib.request import urlopen
from bs4 import BeautifulSoup
import csv
import unicodedata
import pandas as pd

In [83]:
start_url = "https://mailarchive.upatras.gr/search/20091231.220000.00000000@.en.html"

In [84]:
fields=['Author','Date','To','Subject','Message']
filename='testing.csv'
with open(filename,'w',encoding='utf-32', newline='') as csvfile:
    csvwriter=csv.writer(csvfile,delimiter='\t')
    csvwriter.writerow(fields)

In [85]:
def scrapper(url):
    page = urlopen(url, context=ssl.create_default_context(cafile=certifi.where()))
    soup =  BeautifulSoup(page, "html.parser" )
    links=soup.find_all('a')
    urls=[]
    for link in links[3:-3]:
        a=link['href']
        new_url="https://mailarchive.upatras.gr"+a[2:None]
        urls.append(new_url)
    for urli in urls:
        try:
            new_page = urlopen(urli, context=ssl.create_default_context(cafile=certifi.where()))
            new_soup =  BeautifulSoup(new_page, "html.parser" )
            raw_text=new_soup.find('div',{"class": "messageBody"})
            text=raw_text.get_text()
            raw_info=new_soup.find('div',{"class": None})
            info=raw_info.get_text()
            norm_info = unicodedata.normalize('NFKD', info).encode('utf-32', 'ignore').decode('utf-32')
            x=norm_info.split('Author:')
            y=x[-1].split('Date:')
            z=y[-1].split('To:')
            w=z[-1].split('Subject:')
            data=[y[0],z[0],w[0],w[-1],text]
            with open('testing.csv','a',encoding='utf-32', newline='') as csvfile:
                writer = csv.writer(csvfile,delimiter='\t')
                writer.writerow(data)
        except:
            continue
    
    
    

In [86]:
def find_next_url(url):
    page = urlopen(url, context=ssl.create_default_context(cafile=certifi.where()))
    soup =  BeautifulSoup(page, "html.parser" )
    kl = soup.find_all('td', {'align': 'right'})
    next_url=kl[1].find('a')['href']
    next_url='https://mailarchive.upatras.gr/search/'+next_url
    return next_url

In [87]:
url=start_url
for i in range(10):
    scrapper(url)
    url=find_next_url(url)
    
    

In [89]:
df = pd.read_csv('testing.csv', encoding = 'utf-32',sep='\t')

In [91]:
df.head()

,Author,Date,To,Subject,Message
0,Συνέλλη Αικατερίνη,2009-12-23 11:53 UTC,announces,[ANNOUNCES] Ευχές από ΒΚΠ,\nΤο μήνυμα είναι έτοιμο για αποστολή με τα ακ...
1,John Matsoukas,2009-12-23 12:47 UTC,"secretariat, dgiannak, dgrambio, secretary, Π...",[ANNOUNCES] Fw: Ευχές,\n\n\n\n\n\n\n\nΤο Μεταπτυχιακό Πρόγραμμα Σπου...
2,Αντιπρύτανης Οικονομικού Προγραμματισμού κ...,2009-12-23 21:24 UTC,announcements,[ANNOUNCEMENTS] Ευχές,\n\nΗ Πρυτανική Αρχή του Πανεπιστημίου Πατρών ...
3,ΣΥΛΛΟΓΟΣ ΔΕΠ,2009-12-24 16:02 UTC,announcements,[ANNOUNCEMENTS] Ευχές,Εκ μέρους όλων των μελών του Δ.Σ. του συλλόγου...
4,Αντιπρύτανης Οικονομικού Προγραμματισμού κ...,2009-12-27 08:42 UTC,announcements,[ANNOUNCEMENTS] Ενημέρωση για την 62 Σύνοδο...,\n\nΕνημέρωση για την 62η Σύνοδο των Πρυτάνεων...


In [93]:
print(df['Message'][0])


Το μήνυμα είναι έτοιμο για αποστολή με τα ακόλουθα συνημμένα αρχεία ή
συνδέσεις:

LIS_Christmas_Greetings


Σημείωση: Για προστασία κατά των ιών υπολογιστών, τα προγράμματα
ηλεκτρονικού ταχυδρομείου μπορεί να εμποδίζουν την αποστολή ή λήψη ορισμένων
τύπων συνημμένων αρχείων.  Ελέγξτε τις ρυθμίσεις ασφάλειας του ηλεκτρονικού
σας ταχυδρομείου, για να εξακριβώσετε πώς γίνεται ο χειρισμός των
συνημμένων.

